In [27]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate

In [28]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and language in ('C','C++')
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and language in ('C','C++')
    and status='Fixed'
    and a2.actionability=0
    and memory =1
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
# df=df.sort_values('total', ascending=False)
# df.total=df.total.apply(lambda x: '{:,}'.format(x))
# df.i=df.i.astype(str)+"%"
# df.fp=df.fp.astype(str)+"%"
# df.alive=df.alive.astype(str)+"%"
# df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total memory','i':'intentional memory','fp': 'false positive memory','c':'unknown elimination memory','alive':'alive memory'})
# df=df.rename(columns={'total':'Total Memory Alerts','i':'Intentional','fp': 'False Positive','c':'Unknown Elimination','alive':'Alive'})
memory=df
print(tabulate(memory, tablefmt='pipe',headers='keys',showindex=False))

| name        |   total memory |   intentional memory |   false positive memory |   alive memory |   unknown elimination memory |
|:------------|---------------:|---------------------:|------------------------:|---------------:|-----------------------------:|
| Chromium EC |            306 |                 0.33 |                    2.61 |           3.59 |                        63.73 |
| Firefox     |           5448 |                 3.82 |                    9.69 |           8.87 |                        25.11 |
| Kodi        |            682 |                10.56 |                    9.38 |          11.44 |                         9.09 |
| LibreOffice |           3975 |                 0.78 |                   18.39 |           0.08 |                         7.02 |
| Linux       |           7903 |                 1.64 |                    4.02 |          21.08 |                        28.22 |
| OpenCV      |            710 |                 0.56 |                    1.97 |         

In [29]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and language in ('C','C++')
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
intentional=root[root['classification']=='Intentional'].groupby('name')[['cid']].count()
intentional.rename(columns={'cid':'i'},inplace=True)
fp=root[root['classification']=='False Positive'].groupby('name')[['cid']].count()
fp.rename(columns={'cid':'fp'},inplace=True)
alive = root[root['status']=='New'].groupby('name')[['cid']].count()
alive.rename(columns={'cid':'alive'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive])
q='''select name, count(*) as c from alert a
    join project p on a.project_id = p.id
    join memory_error me on a.alert_type_id = me.alert_type_id
    join actionability a2 on a.id = a2.alert_id
    where is_invalid = 0
    and status='Fixed'
    and a2.actionability=0
    and memory is null
    and language in ('C','C++')
    and a2.file_deleted is null
    and a2.file_renamed is null
    and a2.suppression is null
    group by name
    order by name;
    '''
unact=pd.DataFrame(sql.execute(q))
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,intentional,fp,alive,unact])
df.i=round((df.i/df.total)*100,2)
df.fp=round((df.fp/df.total)*100,2)
df.alive=round((df.alive/df.total)*100,2)
df.c=round((df.c/df.total)*100,2)
# df.i=df.i.astype(str)+"%"
# df.fp=df.fp.astype(str)+"%"
# df.alive=df.alive.astype(str)+"%"
# df.c=df.c.astype(str)+"%"
df=df.rename(columns={'total':'total nonmemory','i':'intentional nonmemory','fp': 'false positive nonmemory','c':'unknown elimination nonmemory','alive':'alive nonmemory'})
nonmemory=df
nonmemory

,name,total nonmemory,intentional nonmemory,false positive nonmemory,alive nonmemory,unknown elimination nonmemory
0,Chromium EC,658,8.66,9.57,9.88,43.16
1,Firefox,7496,10.33,5.80,12.47,22.27
2,Kodi,1711,15.08,1.17,16.42,14.03
3,LibreOffice,8007,3.80,2.10,0.05,23.63
4,Linux,11611,5.66,3.81,24.92,22.77
5,OpenCV,1733,7.04,2.71,13.79,33.18
6,Samba,2807,2.21,2.24,21.80,47.17
7,Thunderbird,528,0.57,0.19,46.59,34.85
8,VTK,1356,0.44,1.77,77.88,9.96


In [30]:
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[memory,nonmemory])
df=df.reset_index()
df

,index,name,total memory,intentional memory,false positive memory,alive memory,unknown elimination memory,total nonmemory,intentional nonmemory,false positive nonmemory,alive nonmemory,unknown elimination nonmemory
0,0,Chromium EC,306,0.33,2.61,3.59,63.73,658,8.66,9.57,9.88,43.16
1,1,Firefox,5448,3.82,9.69,8.87,25.11,7496,10.33,5.80,12.47,22.27
2,2,Kodi,682,10.56,9.38,11.44,9.09,1711,15.08,1.17,16.42,14.03
3,3,LibreOffice,3975,0.78,18.39,0.08,7.02,8007,3.80,2.10,0.05,23.63
4,4,Linux,7903,1.64,4.02,21.08,28.22,11611,5.66,3.81,24.92,22.77
5,5,OpenCV,710,0.56,1.97,8.31,41.55,1733,7.04,2.71,13.79,33.18
6,6,Samba,1700,0.82,2.88,18.12,35.53,2807,2.21,2.24,21.80,47.17
7,7,Thunderbird,357,0.28,0.28,31.37,38.94,528,0.57,0.19,46.59,34.85
8,8,VTK,1166,0.09,0.17,86.96,4.80,1356,0.44,1.77,77.88,9.96


In [31]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['intentional memory'], df['intentional nonmemory'])
stat, p, df['intentional memory'].median(), df['intentional nonmemory'].median()

(19.0, 0.03184488756831434, 0.78, 5.66)

In [32]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['false positive memory'], df['false positive nonmemory'])
stat, p, df['false positive memory'].median(), df['false positive nonmemory'].median()

(33.0, 0.2682497346597282, 2.88, 2.24)

In [33]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['alive memory'], df['alive nonmemory'])
stat, p, df['alive memory'].median(), df['alive nonmemory'].median()

(32.0, 0.23996434607297862, 11.44, 16.42)